In [7]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from badass_music.theory.western.scales.chromatic import chromatic_scale_pitch_class_names

In [8]:
track_to_analyze_filename = os.environ['BMPP_HOME'] + '/badass_music/data/Lady Napalm - Kwiky Bridge (Instant Satisfaction Mix).mp3'
#track_to_analyze_filename = os.environ['BMPP_HOME'] + '/badass_music/data/Lady Napalm - Voice in the Distance.mp3'

sampling_rate_for_import = 11025 / 4 # 22050   48000
hop_length_min = 1000
number_of_seconds_max = 2

chroma_function_to_use = librosa.feature.chroma_stft

In [3]:
hop_length_array = np.arange(
    hop_length_min,
    (sampling_rate_for_import * number_of_seconds_max) + hop_length_min,
    hop_length_min,
)

In [9]:
y, sr = librosa.load(
    track_to_analyze_filename,
    sr = sampling_rate_for_import,
)  

In [10]:
y_harmonic, y_percussive = librosa.effects.hpss(y)

In [6]:
if True:
    results_list = []
    for hop_length in hop_length_array:
        results_list.append(
            chroma_function_to_use(
                y = y_harmonic,
                sr = sampling_rate_for_import,
                hop_length = hop_length,
            )
        )
    results_list_per_note_means = np.array([np.mean(x, axis=1) for x in results_list])

ParameterError: hop_length=1000.0 must be a positive integer

In [ ]:
df = pd.DataFrame(results_list_per_note_means, columns = chromatic_scale_pitch_class_names)

In [ ]:
df

In [ ]:
from scipy.stats import entropy
ent = np.array([entropy(x) for x in results_list_per_note_means])
#ent

plt.figure()
plt.scatter(hop_length_array, ent)
plt.show()
plt.close()

In [ ]:
plt.figure()
plt.scatter(hop_length_array, np.std(results_list_per_note_means, axis = 1))
plt.show()
plt.close()

In [ ]:
estimated_tonic = np.argmax(results_list_per_note_means, axis = 1)
estimated_tonic

In [ ]:
estimated_least = np.argmin(results_list_per_note_means, axis = 1)
estimated_least

In [ ]:
for mx, mn in zip(estimated_tonic, estimated_least):
    abs_range = results_list_per_note_means[:, mx] - results_list_per_note_means[:, mn]

plt.figure()
plt.scatter(hop_length_array, abs_range)
plt.show()
plt.close()

In [ ]:
df

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def scale_it(features):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled


features = df.to_numpy()
features_scaled = scale_it(features)

pca = PCA(n_components = 2)
pca.fit(features_scaled)
features_scaled_pca = pca.transform(features_scaled)

features_scaled_pca_scaled = scale_it(features_scaled_pca)

kmeans = KMeans(
    init = 'random',
    n_clusters = 4,
    n_init = 10,
    max_iter = 300,
    random_state = 42
)

kmeans.fit(features_scaled_pca_scaled)

print(kmeans.inertia_)
print()
print(kmeans.cluster_centers_)
print()
print(kmeans.labels_)

df_post_kmeans = df.copy()
df_post_kmeans['hop_length'] = hop_length_array
df_post_kmeans['label_from_kmeans'] = kmeans.labels_

plt.figure()
plt.scatter(features_scaled_pca[:, 0], features_scaled_pca[:, 1], c = kmeans.labels_)
plt.show()
plt.close()

In [ ]:
df_post_kmeans

In [ ]:
df_post_kmeans.groupby('label_from_kmeans')['C'].agg('count')

In [ ]:
plt.figure()
plt.bar(df_post_kmeans.index, df_post_kmeans['label_from_kmeans'], align = 'center')
plt.show()
plt.close()

In [ ]:
if True:
    dft_result_numpy = np.fft.fft(y_harmonic)

amplitudes = np.abs(dft_result_numpy)

In [ ]:
#from badass_music.MIDI.MIDI_number_to import MIDI_note_number_array_to_frequency_array

if True:
    L = y_harmonic.shape[0]

    #
    # https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter24.02-Discrete-Fourier-Transform.html
    #
    T = L / sampling_rate_for_import
    N = L
    n = np.arange(N)

    freq = n / T
    #freq = np.fft.fftfreq(L, d = 1. / sampling_rate_for_import)
    

    X = dft_result_numpy
    N = len(X)
    n = np.arange(N)
    T = N / sampling_rate_for_import
    freq = n / T 

    plt.figure(figsize = (8, 6))
    plt.stem(freq, abs(X), 'b', \
             markerfmt=" ", basefmt="-b")
    plt.xlabel('Freq (Hz)')
    plt.ylabel('DFT Amplitude |X(freq)|')
    plt.show()

    
    if False:
        #print(L)
        #print(y_harmonic.shape)
        #print(T)

        n_half = np.uint64(np.floor( (len(n) - 1) / 2 ))
        indices = np.uint64(np.arange(0, n_half))

        print(indices)
        #print(T)
        print(amplitudes[indices[0:5]])
        print(freq[indices[-5:]])
        #print(freq)

        #how_many = 100
        #plt.figure()
        #plt.stem(freq[0:how_many], abs(dft_result_numpy[0:how_many]), 'b', markerfmt = ' ', basefmt = '-b')
        #plt.show()
        #plt.close()

